# Exploratory Data Analysis (EDA) - Diabetes Dataset
## Comprehensive analysis to understand data distribution, missing values, correlations, and class differences

## 1. Import Libraries and Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configure visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Load the dataset from parquet file
df = pd.read_parquet('./data/final/nhanes_diabetes.parquet')

print("Dataset loaded successfully")
print(f"Shape: {df.shape}")

FileNotFoundError: [Errno 2] No such file or directory: 'dataset.parquet'

## 2. General Dataset Information

In [ ]:
# Display general information about the dataset
print("=" * 80)
print("GENERAL DATASET INFORMATION")
print("=" * 80)

# Dimensions and basic info
print(f"\nDimensions: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"\nData types:\n{df.dtypes}")
print(f"\nMemory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Display first rows
print("\nFirst 5 rows:")
display(df.head())

# Display last rows
print("\nLast 5 rows:")
display(df.tail())

## 3. Missing Data Analysis

In [ ]:
# Analyze missing values across all features
print("=" * 80)
print("MISSING DATA ANALYSIS")
print("=" * 80)

# Count missing values per column
missing_data = df.isnull().sum()
missing_percent = (df.isnull().sum() / len(df)) * 100
missing_df = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': missing_data.values,
    'Missing_Percentage': missing_percent.values
}).sort_values('Missing_Count', ascending=False)

print("\nMissing values per column:")
display(missing_df[missing_df['Missing_Count'] > 0])

# Visualize missing data if any exists
if missing_data.sum() > 0:
    plt.figure(figsize=(12, 6))
    missing_df_sorted = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=True)
    plt.barh(missing_df_sorted['Column'], missing_df_sorted['Missing_Percentage'], color='coral')
    plt.xlabel('Percentage of Missing Data (%)')
    plt.title('Distribution of Missing Values by Variable')
    plt.tight_layout()
    plt.show()
else:
    print("\n✓ No missing data found in the dataset")

## 4. Target Variable Analysis (Diabetes)

In [ ]:
# Analyze the diabetes target variable
print("=" * 80)
print("TARGET VARIABLE ANALYSIS: DIABETES")
print("=" * 80)

# Define target column name - ADJUST IF YOUR COLUMN NAME IS DIFFERENT
target_col = 'diabetes'  # CHANGE THIS IF NEEDED

# Examine unique values
print(f"\nUnique values in '{target_col}':")
print(df[target_col].value_counts().sort_index())

print(f"\nDistribution (%):")
print((df[target_col].value_counts(normalize=True) * 100).sort_index())

# Filter to keep only positive (1) and negative (2) cases
print(f"\nTotal records before filtering: {len(df)}")
df_filtered = df[df[target_col].isin([1, 2])].copy()
print(f"Records after filtering only diabetes (1, 2): {len(df_filtered)}")
print(f"Records removed: {len(df) - len(df_filtered)}")

# Update dataframe with filtered data
df = df_filtered

# Display class distribution after filtering
print(f"\nClass distribution after filtering:")
class_dist = df[target_col].value_counts().sort_index()
print(class_dist)

# Visualize target distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Bar chart - absolute frequency
class_dist.plot(kind='bar', ax=axes[0], color=['skyblue', 'salmon'])
axes[0].set_title('Diabetes Distribution (Absolute Frequency)')
axes[0].set_xlabel('Diabetes Class')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)

# Pie chart - percentage
class_dist.plot(kind='pie', ax=axes[1], autopct='%1.1f%%', colors=['skyblue', 'salmon'])
axes[1].set_title('Diabetes Distribution (Percentage)')
axes[1].set_ylabel('')

plt.tight_layout()
plt.show()

# Calculate and display class imbalance ratio
imbalance_ratio = class_dist.max() / class_dist.min()
print(f"\nImbalance ratio: {imbalance_ratio:.2f}:1")

## 5. Descriptive Statistics for Numeric Variables

In [ ]:
# Compute descriptive statistics for all numeric features
print("=" * 80)
print("DESCRIPTIVE STATISTICS - NUMERIC VARIABLES")
print("=" * 80)

# Select numeric columns (excluding target variable)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if target_col in numeric_cols:
    numeric_cols.remove(target_col)

print(f"\nNumeric variables found: {len(numeric_cols)}")
print(f"Columns: {numeric_cols}\n")

# Calculate comprehensive descriptive statistics
desc_stats = df[numeric_cols].describe().T
desc_stats['variance'] = df[numeric_cols].var()
desc_stats['skewness'] = df[numeric_cols].skew()
desc_stats['kurtosis'] = df[numeric_cols].kurtosis()

print("\nComprehensive descriptive statistics:")
display(desc_stats)

## 6. Distribution of Numeric Variables

In [ ]:
# Visualize distributions of all numeric variables using histograms
print("\nDistributions of Numeric Variables:")

n_cols = len(numeric_cols)
n_rows = (n_cols + 2) // 3

fig, axes = plt.subplots(n_rows, 3, figsize=(15, 4*n_rows))
axes = axes.flatten()

# Create histogram for each numeric variable
for idx, col in enumerate(numeric_cols):
    axes[idx].hist(df[col].dropna(), bins=50, color='steelblue', edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'Distribution: {col}')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frequency')
    # Add mean line
    axes[idx].axvline(df[col].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df[col].mean():.2f}')
    axes[idx].legend()

# Hide empty subplots
for idx in range(n_cols, len(axes)):
    axes[idx].set_visible(False)

plt.tight_layout()
plt.show()

## 7. Outlier Detection

In [ ]:
# Detect outliers using the Interquartile Range (IQR) method
print("=" * 80)
print("OUTLIER DETECTION (IQR Method)")
print("=" * 80)

outliers_summary = {}

# Calculate IQR bounds for each numeric variable
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    # Define outlier bounds (1.5 * IQR is standard)
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identify outliers
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outliers_summary[col] = {
        'count': len(outliers),
        'percent': (len(outliers) / len(df)) * 100,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound
    }

outliers_df = pd.DataFrame(outliers_summary).T.sort_values('count', ascending=False)
print("\nOutliers detected (IQR * 1.5):")
display(outliers_df[outliers_df['count'] > 0])

# Visualize outliers using boxplots
fig, axes = plt.subplots(n_rows, 3, figsize=(15, 4*n_rows))
axes = axes.flatten()

for idx, col in enumerate(numeric_cols):
    axes[idx].boxplot(df[col].dropna())
    axes[idx].set_title(f'Boxplot: {col}')
    axes[idx].set_ylabel(col)
    axes[idx].grid(True, alpha=0.3)

# Hide empty subplots
for idx in range(n_cols, len(axes)):
    axes[idx].set_visible(False)

plt.tight_layout()
plt.show()

## 8. Correlation Analysis

In [ ]:
# Analyze correlations between variables and with target
print("=" * 80)
print("CORRELATION ANALYSIS")
print("=" * 80)

# Calculate correlation matrix
correlation_matrix = df[numeric_cols + [target_col]].corr()

# Display correlations with target variable
print("\nCorrelation with target variable (Diabetes):")
target_corr = correlation_matrix[target_col].drop(target_col).sort_values(ascending=False)
print(target_corr)

# Visualize full correlation matrix as heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            fmt='.2f', square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix - All Variables')
plt.tight_layout()
plt.show()

# Visualize only correlations with target variable
plt.figure(figsize=(10, 6))
target_corr_sorted = target_corr.sort_values()
colors = ['red' if x < 0 else 'green' for x in target_corr_sorted.values]
plt.barh(range(len(target_corr_sorted)), target_corr_sorted.values, color=colors, alpha=0.7)
plt.yticks(range(len(target_corr_sorted)), target_corr_sorted.index)
plt.xlabel('Correlation with Diabetes')
plt.title('Feature Correlation with Target Variable')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
plt.tight_layout()
plt.show()

## 9. Analysis by Class (Diabetes vs. Non-Diabetes)

In [ ]:
# Compare statistics between diabetes classes
print("=" * 80)
print("COMPARISON BETWEEN DIABETES CLASSES")
print("=" * 80)

# Display descriptive statistics grouped by class
print("\nDescriptive statistics by class:")
grouped_stats = df.groupby(target_col)[numeric_cols].describe().T
display(grouped_stats)

# Compare mean values between groups
print("\nComparison of means by class:")
comparison = df.groupby(target_col)[numeric_cols].mean()
display(comparison)

# Visualize differences using boxplots by class
fig, axes = plt.subplots(n_rows, 3, figsize=(15, 4*n_rows))
axes = axes.flatten()

for idx, col in enumerate(numeric_cols):
    df.boxplot(column=col, by=target_col, ax=axes[idx])
    axes[idx].set_title(f'{col} by Diabetes Class')
    axes[idx].set_xlabel('Diabetes')
    axes[idx].set_ylabel(col)
    plt.sca(axes[idx])
    plt.xticks([1, 2], ['No (1)', 'Yes (2)'])

# Remove the default title
plt.suptitle('')
# Hide empty subplots
for idx in range(n_cols, len(axes)):
    axes[idx].set_visible(False)

plt.tight_layout()
plt.show()

## 10. Statistical Tests (Group Differences)

In [ ]:
# Perform t-tests to identify significant differences between groups
print("=" * 80)
print("STUDENT'S T-TEST - Difference Between Groups")
print("=" * 80)

# Separate data by class
group_1 = df[df[target_col] == 1][numeric_cols]
group_2 = df[df[target_col] == 2][numeric_cols]

# Perform t-test for each variable
print("\nStudent's t-test (H0: means are equal):")
t_test_results = {}

for col in numeric_cols:
    t_stat, p_value = stats.ttest_ind(group_1[col].dropna(), group_2[col].dropna())
    t_test_results[col] = {'t_statistic': t_stat, 'p_value': p_value}

t_test_df = pd.DataFrame(t_test_results).T.sort_values('p_value')
t_test_df['Significant (α=0.05)'] = t_test_df['p_value'] < 0.05

print("\nT-test results (sorted by p-value):")
display(t_test_df)

## 11. Categorical Variables Analysis

In [ ]:
# Analyze categorical variables if any exist
print("=" * 80)
print("CATEGORICAL VARIABLES ANALYSIS")
print("=" * 80)

# Identify categorical columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

if categorical_cols:
    print(f"\nCategorical variables found: {len(categorical_cols)}")
    print(f"Columns: {categorical_cols}\n")
    
    # Display value counts for each categorical variable
    for col in categorical_cols:
        print(f"\n{col}:")
        print(f"Unique values: {df[col].nunique()}")
        print(df[col].value_counts())
else:
    print("\nNo categorical variables found")

## 12. EDA Summary and Recommendations

In [ ]:
# Generate comprehensive EDA summary report
print("=" * 80)
print("EDA SUMMARY")
print("=" * 80)

# Create summary report with key findings
summary_report = f"""
DIMENSIONS AND STRUCTURE:
  • Total records: {len(df):,}
  • Total variables: {df.shape[1]}
  • Numeric variables: {len(numeric_cols)}
  • Categorical variables: {len(categorical_cols)}

MISSING DATA:
  • Total percentage of missing data: {(df.isnull().sum().sum() / (len(df) * df.shape[1])) * 100:.2f}%
  • Columns with missing values: {missing_data.sum()}

TARGET VARIABLE (Diabetes):
  • Class 1 (No Diabetes): {class_dist.get(1, 0):,} records ({(class_dist.get(1, 0)/len(df)*100):.1f}%)
  • Class 2 (Yes Diabetes): {class_dist.get(2, 0):,} records ({(class_dist.get(2, 0)/len(df)*100):.1f}%)
  • Imbalance ratio: {imbalance_ratio:.2f}:1

OUTLIERS:
  • Variables with outliers: {sum(1 for x in outliers_df['count'] if x > 0)}
  • Total outliers detected: {outliers_df['count'].sum():.0f}

CORRELATION:
  • Top 5 variables most correlated with Diabetes:
{target_corr.head(5).to_string()}
"""

print(summary_report)

# Save summary report to file
with open('eda_summary_report.txt', 'w') as f:
    f.write(summary_report)

print("\n✓ Report saved to 'eda_summary_report.txt'")

## Important Notes

1. **Adjust target variable name**: If your diabetes column has a different name, update `target_col` in section 4.

2. **Handling missing data**: Based on the results, you can:
   - Remove records with too many missing values
   - Impute with mean/median
   - Use advanced techniques (KNN, iterative imputer)

3. **Outliers**: Do not automatically remove them. Analyze whether they are errors or legitimate values.

4. **Class imbalance**: If significant imbalance exists, consider:
   - Oversampling (SMOTE)
   - Undersampling
   - Class weights in models

5. **Next steps**:
   - Feature engineering
   - Feature selection
   - Normalization/Scaling
   - Predictive modeling